# This is Nifty 500 Correlation Algorithm

## Imports

In [1]:
import os
import pandas as pd
import pickle
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader.data as web

from sklearn import neighbors

from datetime import date
# from nsetools import Nse
from matplotlib import style
from nsepy.history import get_history

from sklearn import svm
from sklearn.model_selection import train_test_split
from collections import Counter

from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 

style.use('ggplot')

## SAVE THE NSE_TICKER_DATA

In [ ]:
def save_nse_500_tickers():
    list_of_stock_codes = list(pd.read_csv('nifty_list/ind_nifty500list.csv')['Symbol'])
    return list_of_stock_codes

def save_nse_200_tickers():
    list_of_stock_codes = list(pd.read_csv('nifty_list/ind_nifty200list.csv')['Symbol'])
    list_of_stock_codes.remove('ACC')
    return list_of_stock_codes

def save_nse_100_tickers():
    list_of_stock_codes = list(pd.read_csv('nifty_list/ind_nifty100list.csv')['Symbol'])
    return list_of_stock_codes

def save_nse_50_tickers():
    list_of_stock_codes = list(pd.read_csv('nifty_list/ind_nifty50list.csv')['Symbol'])
    return list_of_stock_codes

In [ ]:
# get_history(symbol="ADANIPORTS", start=date(2015,1,1), end=date(2015,1,31))
# web.DataReader("ADANIPORTS" + ".NS", 'yahoo', date(2021,1,1), date(2021,5,31))

## Fetch the Historical Data from NSEPY

In [ ]:
def get_data_from_nsepy(ticker_fetch_function):
    
    # Create a directory if it does not exist.
    if not os.path.exists(nse_directory):
        os.makedirs(nse_directory)
    
    tickers = ticker_fetch_function
    
    
    start = dt.datetime(2016, 1, 1)
    end = dt.datetime(2021, 5, 19)

    for ticker in tickers:
        print ("Feching the", ticker)
        if not os.path.exists(nse_directory + '/{}.csv'.format(ticker)):
            
            try:
                df_nse = get_history(symbol=ticker, start=start, end=end)
#                 df_nse = web.DataReader(ticker + ".NS", 'yahoo', start, end)
            except Exception as e:
                continue
            
            df_nse.to_csv(nse_directory + '/{}.csv'.format(ticker))
            print('Dont have {}'.format(ticker))
        else:
            print('Already have {}'.format(ticker))
        

In [ ]:
nse_directory = 'nse_data_50'
get_data_from_nsepy(save_nse_50_tickers())

## Compile the data into one Adjusted close of NIFTY500 Index 

In [ ]:
def compile_data(ticker_fetch_function):
    
    tickers = ticker_fetch_function
   
    
    # Remove invaid stock quotes.
    if tickers in ['ASTERDM', 'FEDERALBNK', 'ACC', '3MINDIA']:
        tickers.remove('ASTERDM')
        tickers.remove('FEDERALBNK')
        tickers.remove('ACC')
        tickers.remove('3MINDIA')
        
    
    main_df = pd.DataFrame()
    
    for count, ticker in enumerate(tickers):
        df = pd.read_csv('nse_data_50/{}.csv'.format(ticker))
        
        df.set_index('Date', inplace = True)
        print(df.columns)
        
        columns_to_be_dropped = ['Symbol', 'Series', 'Prev Close', 'Open', 
                                 'High', 'Low', 'Last', 'VWAP', 'Volume', 
                                 'Turnover', 'Trades', 'Deliverable Volume',
                                 '%Deliverble']

#         columns_to_be_dropped = ['Open', 'High', 'Low', 'Volume', 'Close']
        
        df.rename(columns = {'Close' : ticker}, inplace = True)
        df.drop(columns_to_be_dropped, 1, inplace = True)
        df.fillna(0, inplace=True)
        
        print(df.columns)
        
        if main_df.empty:
            main_df = df
            print(df)
        else:
            print(df)
            main_df = main_df.join(df, how='outer')
        
        if count % 10 == 0:
            print(count)
    
    print(main_df.head())
    main_df.to_csv('nifty50_joined_closes.csv')

In [ ]:
compile_data(save_nse_50_tickers())

## Visualization of Co-relation among companies from nifty-50

In [ ]:
def visualize_data():
    df = pd.read_csv('nifty50_joined_closes.csv')
    df_corr = df.corr()
    df_corr.to_csv('correlation_data')
    print (df_corr.head())
    
    data = df_corr.values
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    
    heatmap = ax.pcolor(data, cmap='Spectral')
    fig.colorbar(heatmap)
    ax.set_xticks(np.arange(data.shape[0]) + 0.5, minor=False)
    ax.set_yticks(np.arange(data.shape[1]) + 0.5, minor=False)
    
    ax.invert_yaxis()
    ax.xaxis.tick_top()
    
    column_labels = df_corr.columns
    row_labels = df_corr.index
    
    ax.set_xticklabels(column_labels)
    ax.set_yticklabels(row_labels)
    plt.xticks(rotation = 90)
    heatmap.set_clim(-1,1)
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_data()

In [ ]:
# pd.read_csv("correlation_data")

## Process data for label and create target and featuresets

In [ ]:
def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('nifty50_joined_closes.csv', index_col = 0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)
    
    for i in range(1, hm_days+1):
        df['{}_{}d'.format(ticker, i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]
        
    df.fillna(0, inplace= True)
    return tickers, df

In [ ]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.030
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0

In [ ]:
def extract_fetauresets(ticker):
    tickers, df = process_data_for_labels(ticker)
    
    df['{}_target'.format(ticker)] = list(map (buy_sell_hold, df['{}_1d'.format(ticker)],
                                                              df['{}_2d'.format(ticker)],
                                                              df['{}_3d'.format(ticker)],
                                                              df['{}_4d'.format(ticker)],
                                                              df['{}_5d'.format(ticker)],
                                                              df['{}_6d'.format(ticker)],
                                                              df['{}_7d'.format(ticker)]))
    
    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
#     print('Data Spread: ', Counter(str_vals))
    df.fillna(0, inplace=True)
    
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    
    df_vals = df[[ticker for ticker in tickers]]
    df_vals = df_vals.replace([np.inf, -np.inf], np.nan)
    df_vals.fillna(0, inplace=True)
    
    X = df_vals.values
    y = df['{}_target'.format(ticker)].values
    
    return X, y, df

## Finally, Perform your Algorithm's Corelation on different Classifiers

In [ ]:
def do_ml(ticker):
    X, y, df = extract_fetauresets(ticker)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
    
    clf = RandomForestClassifier()
    
    estimator = []
    estimator.append(('LR',  
                  LogisticRegression(solver ='lbfgs',  
                                     max_iter = 80, multi_class = "auto"))) 
    estimator.append(('SVC', SVC(gamma ='auto', probability = True))) 
    estimator.append(('DTC', DecisionTreeClassifier())) 
    
#     clf = VotingClassifier(estimators = estimator, voting ='soft')
    
    clf.fit(X_train, y_train)
    
    confidence = clf.score(X_test, y_test)
    print("Accuracy", confidence)
    
    predictions = clf.predict(X_test)
    
    print("Predicted Spread: ", Counter(predictions))
    
    return(confidence)

In [ ]:
X, y, df = extract_fetauresets("ADANIPORTS")

In [ ]:
df

##  Buy sell hold data on Top 50 stocks.

In [ ]:
tickers = save_nse_50_tickers()
hold_buy_sell_df  = pd.DataFrame()
for i in tickers:
    X, y, df = extract_fetauresets(i)
    hold_buy_sell_df[i] = df.loc[df.index > '2020-06-07'][i+"_target"]

In [ ]:
hold_buy_sell_df.loc['2021-05-04']

In [ ]:
do_ml("TCS")

# Analyzing the peer stocks in IT

In [ ]:
# Analysing your Competitors Stocks
PEER_STOCKS = ["TCS", "INFY", "WIPRO", "HCLTECH"]
main_df = pd.DataFrame()

for ticker in PEER_STOCKS:
    print(ticker)
    df = pd.read_csv('nse_data_200/{}.csv'.format(ticker))

    # Compile the data over a interval
    df = df[(df['Date'] > '2021-1-1') & (df['Date'] <= '2021-5-10')]

    df.set_index('Date', inplace = True)

    columns_to_be_dropped = ['Symbol', 'Series', 'Prev Close', 'Open', 
                             'High', 'Low', 'Last', 'VWAP', 'Volume', 
                             'Turnover', 'Trades', 'Deliverable Volume',
                             '%Deliverble']

    df.rename(columns = {'Close' : ticker}, inplace = True)
    df.drop(columns_to_be_dropped, 1, inplace = True)
    df.fillna(0, inplace=True)

    if main_df.empty:
        main_df = df
    else:
        main_df = main_df.join(df, how='outer')


In [ ]:
pct_comp = main_df.pct_change()
corr = pct_comp.corr()

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
scatter_matrix(pct_comp, diagonal='kde', figsize=(10, 10))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style

# Adjusting the size of matplotlib
import matplotlib as mpl
mpl.rc('figure', figsize=(8, 7))
mpl.__version__

# Adjusting the style of matplotlib
style.use('ggplot')


In [ ]:
plt.imshow(corr, cmap='hot', interpolation='none')
plt.colorbar()
plt.xticks(range(len(corr)), corr.columns)
plt.yticks(range(len(corr)), corr.columns);
